## Keras Tuner
- 최적 하이퍼파라미터 세트를 선택하는데 도움을 주는 라이브러리
- ML 성능에 직접적인 영향을 줌

In [3]:
import tensorflow as tf
from tensorflow import keras

import IPython

#!pip install -q -U keras-tuner
import kerastuner as kt

## Data Load
- keras tuner를 이용해 fashion MNIST 의류 이미지 분류 ML 모델에 가장 적합한 하이퍼 파라미터를 찾음

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [5]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Model Define

In [6]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])

  return model

## Tuning

In [7]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

In [8]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [9]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 26s]
val_accuracy: 0.8822000026702881

Best val_accuracy So Far: 0.890999972820282
Total elapsed time: 00h 05m 35s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 384 and the optimal learning rate for the optimizer
is 0.001.



## Retraining
##### apply hyperparameter

In [10]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4771 - accuracy: 0.8299 - val_loss: 0.4221 - val_accuracy: 0.8481
Epoch 2/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3614 - accuracy: 0.8678 - val_loss: 0.3785 - val_accuracy: 0.8652
Epoch 3/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3238 - accuracy: 0.8814 - val_loss: 0.3537 - val_accuracy: 0.8739
Epoch 4/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3022 - accuracy: 0.8884 - val_loss: 0.3485 - val_accuracy: 0.8743
Epoch 5/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2818 - accuracy: 0.8956 - val_loss: 0.3305 - val_accuracy: 0.8779
Epoch 6/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2672 - accuracy: 0.9001 - val_loss: 0.3249 - val_accuracy: 0.8850
Epoch 7/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2536 - accuracy: 0.9046 - val_loss: 0.3273 - val_accuracy: